# SPCS Networking Connectivity Test: AWS S3

**Note: This Notebook should be run in an SPCS Container for testing to be valid**

1. Check that the Network Rule structure matches your requirements.
2. Run the Connectivity Test if your EAI is already configured.
3. Use the Cells to prepare the EAI if needed, then rerun the test.

In [ ]:
# AWS S3 Connectivity Test Configuration

# Details for your AWS S3 region and bucket
AWS_S3_REGION = "us-west-2"
AWS_S3_BUCKET = "my-bucket"

# Hostname and URL construction
AWS_S3_API_HOST = f"{AWS_S3_BUCKET}.s3.{AWS_S3_REGION}.amazonaws.com"
AWS_S3_API_URL = f"https://{AWS_S3_API_HOST}/"

# This role will be used to create the EAI and other objects if necessary
IMPLEMENTATION_ROLE = "ACCOUNTADMIN"

print(f"Configuration:")
print(f"AWS S3 region: {AWS_S3_REGION}")
print(f"AWS S3 bucket: {AWS_S3_BUCKET}")
print(f"AWS S3 API host: {AWS_S3_API_HOST}")
print("\nReady to test connectivity...")

In [ ]:
import requests
from urllib.parse import urlparse

def test_connectivity(url, description):
    """Test HTTP connectivity to a given URL"""
    try:
        print(f"\n🔍 Testing {description}: {url}")

        # Simple HTTP test - let EAI handle the routing
        response = requests.get(url, timeout=10, allow_redirects=True)
        print(f"  ✅ HTTP Response: {response.status_code}")

        # Any response means connectivity is working
        return True

    except requests.exceptions.ConnectionError as e:
        print(f"  ❌ Connection Error: {e}")
        return False
    except requests.exceptions.Timeout as e:
        print(f"  ❌ Timeout Error: {e}")
        return False
    except Exception as e:
        print(f"  ❌ Error: {e}")
        return False

# Run connectivity tests
print("=" * 60)
print("AWS S3 CONNECTIVITY TEST")
print("=" * 60)

results = {}
results['aws_s3_api'] = test_connectivity(AWS_S3_API_URL, "AWS S3 API")

print("\n" + "=" * 60)
print("CONNECTIVITY TEST SUMMARY")
print("=" * 60)

all_passed = True
for service, passed in results.items():
    status = "✅ PASS" if passed else "❌ FAIL"
    print(f"{service.upper()}: {status}")
    if not passed:
        all_passed = False

if all_passed:
    print(f"\n🎉 All connectivity tests PASSED!")
    print("Your SPCS environment can reach all required AWS S3 endpoints.")
    print("You can proceed with AWS S3 integration.")
else:
    print(f"\n⚠️  Some connectivity tests FAILED!")
    print("You need to configure Snowflake External Access Integration (EAI).")
    print("Please run the EAI setup cells below to resolve connectivity issues.")

If Connectivity testing fails, you can use the Cells below to prepare and implement an EAI suitable for AWS S3 access.

In [ ]:
-- Create Network Rules for AWS S3 connectivity
-- Run this cell if connectivity tests failed

-- Network rule for AWS S3 API
CREATE OR REPLACE NETWORK RULE aws_s3_connectivity_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('{{AWS_S3_API_HOST}}');

SHOW NETWORK RULES LIKE 'aws_s3_%';

In [ ]:
use role {{IMPLEMENTATION_ROLE}};
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION aws_s3_eai
  ALLOWED_NETWORK_RULES = (aws_s3_connectivity_rule)
  ENABLED = TRUE
  COMMENT = 'External Access Integration for AWS S3 connectivity';

  SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'aws_s3_eai';

In [ ]:
ALTER NOTEBOOK EAI_AWS_S3
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('AWS_S3_EAI');

After Creating and Setting the EAI on the Notebook, run the Connectivity test again.

*NOTE: Remember to restart your compute session*

In [ ]:
-- Check if network rules exist
SHOW NETWORK RULES LIKE 'aws_s3_%';

In [ ]:
-- Check if EAI exists
SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'aws_s3_eai';